<a href="https://colab.research.google.com/github/kapplan/Advanced-R-Programming/blob/main/GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install earthengine-api


In [7]:
import ee
from google.colab import auth

ee.Authenticate()

True

In [10]:
ee.Initialize(project='ee-basakaplan503')

In [11]:
# TEST
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [24]:
# Load the countries dataset and filter by Poland's name
poland = ee.FeatureCollection('FAO/GAUL/2015/level0') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Poland'))

# Visualize the borders (optional)
print('Poland borders loaded')

Poland borders loaded


In [29]:
# Extract the geometry from the FeatureCollection
poland_geometry = poland.geometry()

In [34]:
# Function to mask clouds and shadows
def mask_landsat8_sr(image):
    cloud_shadow_bit_mask = 1 << 4
    clouds_bit_mask = 1 << 3
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) \
            .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return image.updateMask(mask)


In [38]:
# Function to calculate NDVI and mask non-vegetation areas
def calculate_and_mask_ndvi(image):
    # NDVI calculation
    sr_image = image.multiply(0.0000275).add(-0.2)
    ndvi = sr_image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

    # Mask out areas where NDVI is less than 0.4 (higher threshold for vegetation)
    ndvi_masked = ndvi.updateMask(ndvi.gte(0.4))
    return ndvi_masked

In [41]:

# Load the updated GHSL dataset to mask urban areas
urban_areas = ee.Image('JRC/GHSL/P2016/BUILT_V1') \
    .select('built').clip(poland_geometry)

# Load MODIS Water Mask to mask water bodies
water_bodies = ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24') \
    .select('water_mask').clip(poland_geometry)



In [42]:

# Function to mask urban areas using the updated GHSL
def mask_urban_areas(image):
    urban_mask = urban_areas.lt(1)  # Areas with built environment set to 1
    return image.updateMask(urban_mask)

# Function to mask water bodies using MODIS
def mask_water_bodies(image):
    water_mask = water_bodies.neq(1)  # Water bodies are coded as 1
    return image.updateMask(water_mask)

# Function to calculate NDVI and mask non-vegetation areas
def calculate_and_mask_ndvi(image):
    # NDVI calculation
    sr_image = image.multiply(0.0000275).add(-0.2)
    ndvi = sr_image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

    # Mask out areas where NDVI is less than 0.4 (higher threshold for vegetation)
    ndvi_masked = ndvi.updateMask(ndvi.gte(0.4))

    # Mask out urban areas and water bodies
    ndvi_masked = mask_urban_areas(ndvi_masked)
    ndvi_masked = mask_water_bodies(ndvi_masked)

    return ndvi_masked

# Load Landsat 8 data, filter by bounds and date, and calculate NDVI
landsat8_c2 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(poland_geometry) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .map(calculate_and_mask_ndvi)

# Get the mean NDVI for the year, clipped to Poland's geometry
ndvi_mean_c2 = landsat8_c2.mean().clip(poland_geometry)

# Visualization parameters for NDVI
ndvi_vis_c2 = {
    'min': 0.4,  # Set a higher minimum for vegetation
    'max': 0.8,  # Adjust the upper bound for vegetation
    'palette': ['white', 'green']
}

# Generate the URL for the NDVI image
url_c2 = ndvi_mean_c2.getThumbURL({
    'min': 0.4,
    'max': 0.8,
    'palette': ['white', 'green'],
    'dimensions': '1024x1024',  # Specify larger dimensions
    'region': poland_geometry   # Define the region of interest as the geometry
})

print('NDVI Image URL for Collection 2:', url_c2)

EEException: Image.load: Image asset 'JRC/GHSL/P2016/BUILT_V1' not found (does not exist or caller does not have access).

In [36]:
# Load Landsat 8 data, filter by bounds and date, and calculate NDVI
landsat8_c2 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(poland_geometry) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .map(calculate_and_mask_ndvi)


In [37]:
# Get the mean NDVI for the year, clipped to Poland's geometry
ndvi_mean_c2 = landsat8_c2.mean().clip(poland_geometry)

# Visualization parameters for NDVI
ndvi_vis_c2 = {
    'min': 0.2,  # Set a lower bound for vegetation
    'max': 0.8,  # Adjust the upper bound for vegetation
    'palette': ['white', 'green']
}

# Generate the URL for the NDVI image
url_c2 = ndvi_mean_c2.getThumbURL({
    'min': 0.2,
    'max': 0.8,
    'palette': ['white', 'green'],
    'dimensions': '1024x1024',  # Specify larger dimensions
    'region': poland_geometry   # Define the region of interest as the geometry
})

print('NDVI Image URL for Collection 2:', url_c2)

NDVI Image URL for Collection 2: https://earthengine.googleapis.com/v1/projects/ee-basakaplan503/thumbnails/86e88371ec8be3c1494a2cdd139b0172-54aecef62a3794439c908823dde32adb:getPixels
